In [49]:
from graph_tool.all import *
from numpy.random import randint
from numpy.random import random
from numpy.random import *
from IPython.display import Image #import to displaz images
from matplotlib import *
from gi.repository import Gtk, Gdk, GdkPixbuf, GObject, GLib
import sys, os, os.path




In [50]:
g = Graph(directed=False)

#definition of vertex properties

S = [1, 1, 1, 1]           # White color

I = [0, 0, 0, 1]           # Black color

R = [0.5, 0.5, 0.5, 1.]    # Grey color (will not actually be drawn)

state = g.new_vertex_property("vector<double>")
age=g.new_vertex_property("int")
vac=g.new_vertex_property("bool")
removed = g.new_vertex_property("bool")
newly_infected = g.new_vertex_property("bool")



agegroups=20
agegrouplists = [[] for i in range(1, agegroups+1)]


#definition of edge properties

#for not I think an edge just exists so there shouldnt be any edge properties


for v in g.vertices():
    i=randint(1,age_groups)
    vprop_age[v] = i
    

x = 0.001    # spontaneous outbreak probability

r = 0.1      # I->R probability

s = 0.01     # R->S probability






random(g.num_vertices())

random(g.num_vertices())

In [51]:
##THIS IS THE DEFINITION OF VERTICES AND ESGE PROPERTIES
#insert random vertices (nodes)
g.add_vertex(100)

# insert some random links
for s,t in zip(randint(0, 100, 100), randint(0, 100, 100)):
    g.add_edge(g.vertex(s), g.vertex(t))
# layout positions

pos = sfdp_layout(g)

for i in g.vertices():
    v = g.vertex(i) 
    state[v] = S
    
for i in g.vertices():
    v = g.vertex(i)
    a =randint(1,agegroups)
    age[v] = a
    agegrouplists[a].append(v)

if 1 == 2:
    for i in g.vertices():
        v = g.vertex(i)
        vac[v] = False

        
graph_draw(g, vertex_text=g.vertex_index, output="two-nodes.png")
fig = Image(filename=('two-nodes.png'))
fig


# If True, the frames will be dumped to disk as images.

offscreen = sys.argv[1] == "offscreen" if len(sys.argv) > 1 else False

max_count = 500

if offscreen and not os.path.exists("./frames"):

    os.mkdir("./frames")


# This creates a GTK+ window with the initial graph layout

if not offscreen:

    win = GraphWindow(g, pos, geometry=(500, 400),

                      edge_color=[0.6, 0.6, 0.6, 1],

                      vertex_fill_color=state,

                      vertex_halo=newly_infected,

                      vertex_halo_color=[0.8, 0, 0, 0.6])

else:

    count = 0

    win = Gtk.OffscreenWindow()

    win.set_default_size(500, 400)

    win.graph = GraphWidget(g, pos,

                            edge_color=[0.6, 0.6, 0.6, 1],

                            vertex_fill_color=state,

                            vertex_halo=newly_infected,

                            vertex_halo_color=[0.8, 0, 0, 0.6])

    win.add(win.graph)

In [52]:
def update_state():

    newly_infected.a = False

    removed.a = False


    # visit the nodes in random order

    vs = list(g.vertices())

    shuffle(vs)

    for v in vs:

        if state[v] == I:

            if random() < r:

                state[v] = R

        elif state[v] == S:

            if random() < x:

                state[v] = I

            else:

                ns = list(v.out_neighbors())

                if len(ns) > 0:

                    w = ns[randint(0, len(ns))]  # choose a random neighbor

                    if state[w] == I:

                        state[v] = I

                        newly_infected[v] = True

        elif random() < s:

            state[v] = S

        if state[v] == R:

            removed[v] = True


    # Filter out the recovered vertices

    g.set_vertex_filter(removed, inverted=True)


    # The following will force the re-drawing of the graph, and issue a

    # re-drawing of the GTK window.

    win.graph.regenerate_surface()

    win.graph.queue_draw()


    # if doing an offscreen animation, dump frame to disk

    if offscreen:

        global count

        pixbuf = win.get_pixbuf()

        pixbuf.savev(r'./frames/sirs%06d.png' % count, 'png', [], [])

        if count > max_count:

            sys.exit(0)

        count += 1


    # We need to return True so that the main loop will call this function more

    # than once.

    return True


In [53]:
cid = GLib.idle_add(update_state)


# We will give the user the ability to stop the program by closing the window.

win.connect("delete_event", Gtk.main_quit)


# Actually show the window, and start the main loop.

win.show_all()

Gtk.main()

In [54]:
g.list_properties()

In [55]:
g.save("my_graph.xml.gz")
g2 = load_graph("my_graph.xml.gz")